In [1]:
import openai
import pandas as pd
import ast  
import re

In [2]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [3]:
def extract_valid_list(response_text):
    
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text)  
    response_text = response_text.strip("`")  

    match = re.search(r"\[.*?\]", response_text, re.DOTALL)

    if match:
        list_text = match.group(0)  
        try:
            extracted_list = ast.literal_eval(list_text)  # Safely evaluate
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception:
            pass  
    
    return []  

In [4]:
def generate_ner_tags(sentence, client):
    system_prompt = (
        "You are a large language model that outputs only valid Python lists. "
        "No explanations. No additional text. No Markdown formatting."
    )
    
    user_prompt = f"""
    You are a highly intelligent language model trained for natural language understanding and named entity recognition.
    Your task is to assign the most suitable ner_tags to every token in the following text based on the provided list of possible ner_tags.
    
    Assign NER tags from the set:
    ['B-Further Specification', 'I-Further Specification', 'O', 'I-XOR Gateway',
     'B-XOR Gateway', 'B-Activity Data', 'B-Condition Specification', 'I-Actor',
     'B-AND Gateway', 'I-Activity','B-Activity', 'I-Activity Data',
     'I-Condition Specification', 'I-AND Gateway', 'B-Actor']
    to each token in:
    {sentence}

    Output ONLY a Python-style list of NER tags, e.g.:
    ['O', 'O', 'B-Actor', 'I-Actor']

    No explanations. No additional text. Do not include ```python or any Markdown formatting.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
        max_tokens=1024
    )

    response_text = response.choices[0].message.content.strip()
    return extract_valid_list(response_text)

In [5]:
def process_dataset(input_file, output_file, api_key_file):
    api_key = get_api_key(api_key_file)
    client = openai.OpenAI(api_key=api_key)  
    
    dataset = pd.read_csv(input_file)

    def process_row(row):
        sentence = " ".join(ast.literal_eval(row["tokens"]))
        try:
            return generate_ner_tags(sentence, client)
        except Exception:
            return []  

    dataset["gpt4o_ner_tags"] = dataset.apply(process_row, axis=1)
    
    dataset.to_csv(output_file, index=False)

In [6]:
if __name__ == "__main__":
    input_file = "PET-Customized.csv"
    output_file = "PET-Customized-GPT4o.csv"
    api_key_file = "./API_Key/OpenAI.txt"

    process_dataset(input_file, output_file, api_key_file)